In [1]:
# loading pretrained inception V3 model for generating deep dream images

import keras
from keras.applications import inception_v3
from keras import backend as K

# We will not be training our model,
# so we use this command to disable all training-specific operations
K.set_learning_phase(0)

# Build the InceptionV3 network.
# The model will be loaded with pre-trained ImageNet weights.
model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

Using TensorFlow backend.
W0716 10:32:02.597120 140515978704640 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 10:32:02.598558 140515978704640 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:442: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0716 10:32:02.600609 140515978704640 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3543: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0716 10:32:02.740610 140515978704640 deprecation_wrapper.py:118] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backe

In [2]:
# setting the deepdream config, i.e. what layers should contribute to the final image
# this is a somewhat arbitraty selection but can be changed and experimented with

# Dict mapping layer names to a coefficient quantifying how much the layer's activation
# will contribute to the loss we will seek to maximize.
# Note that these are layer names as they appear in the built-in InceptionV3 application.
# You can list all layer names using `model.summary()`.
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [3]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [4]:
# defininig a loss by the weighted sum of L2 norm of the activations of the layers defined above

# Get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Define the loss.
loss = K.variable(0.) #instantiating a variable with value 0
for layer_name in layer_contributions:
    # Add the L2 norm of the features of a layer to the loss.
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    # We avoid border artifacts by only involving non-border pixels in the loss.
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

W0716 10:32:07.759079 140515978704640 deprecation.py:506] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1205: calling reduce_prod_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W0716 10:32:07.763297 140515978704640 deprecation.py:506] From /home/joel/anaconda3/envs/tensorflow-gpu-new/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1188: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W0716 10:32:07.766356 140515978704640 variables.py:2569] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


In [5]:
layer_dict

{'activation_1': <keras.layers.core.Activation at 0x7fcc1a607358>,
 'activation_10': <keras.layers.core.Activation at 0x7fcc08889b38>,
 'activation_11': <keras.layers.core.Activation at 0x7fcc0884f9e8>,
 'activation_12': <keras.layers.core.Activation at 0x7fcc087b8b70>,
 'activation_13': <keras.layers.core.Activation at 0x7fcc0871ab70>,
 'activation_14': <keras.layers.core.Activation at 0x7fcc086fea20>,
 'activation_15': <keras.layers.core.Activation at 0x7fcc08663ef0>,
 'activation_16': <keras.layers.core.Activation at 0x7fcc0862a6d8>,
 'activation_17': <keras.layers.core.Activation at 0x7fcc08610710>,
 'activation_18': <keras.layers.core.Activation at 0x7fcc08576400>,
 'activation_19': <keras.layers.core.Activation at 0x7fcc084d97f0>,
 'activation_2': <keras.layers.core.Activation at 0x7fcc1a5a7860>,
 'activation_20': <keras.layers.core.Activation at 0x7fcc084bce48>,
 'activation_21': <keras.layers.core.Activation at 0x7fcc08482470>,
 'activation_22': <keras.layers.core.Activation at

In [6]:
# Same as in making visible the layers activation in chapter 5, we define a gradient ascent process.

# This holds our generated image
dream = model.input

# Compute the gradients of the dream with regard to the loss.
grads = K.gradients(loss, dream)[0]

# Normalize gradients --> important trick
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Set up function to retrieve the value
# of the loss and gradients given an input image.
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [8]:
# defining auxiliary numpy functions for the deepdream algorithm

import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
# We start by defining a list of scales at which to process the image.
# Then we process a small image with the deepdream algorithm, scale it up by 40% and go on like this.

import numpy as np

# Playing with these hyperparameters will also allow you to achieve new effects

step = 0.01  # Gradient ascent step size
num_octave = 3  # Number of scales at which to run gradient ascent
octave_scale = 1.4  # Size ratio between scales
iterations = 20  # Number of ascent steps per scale

# If our loss gets larger than 10,
# we will interrupt the gradient ascent process, to avoid ugly artifacts
max_loss = 10.

# Fill this to the path to the image you want to use
base_image_path = '/home/ubuntu/data/original_photo_deep_dream.jpg'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/data/original_photo_deep_dream.jpg'